# TP3 - Graphs in ML

Student: Lucas Gerretsen

Instructor: Pierre Perrault

Deadline: 26/12/2018

## Q2

We implement *incremental k-centers* to solve an online semi-supervised problem of face recognition.

In [1]:
from importlib import reload
import Online_SSL.func as ssl
reload(ssl)

<module 'Online_SSL.func' from '/home/lucas/Documents/MVA/Graphs/TP3/code_material/Online_SSL/func.py'>

In [12]:
ssl.create_user_profile("lucas", faces_path = "./data/")

Profile found with 8 images.
Image 9 saved at ./data/faces/lucas/img_9.bmp
Image 10 saved at ./data/faces/lucas/img_10.bmp
Image 11 saved at ./data/faces/lucas/img_11.bmp
Image 12 saved at ./data/faces/lucas/img_12.bmp
Image 13 saved at ./data/faces/lucas/img_13.bmp


In [2]:
ssl.online_face_recognition(['alice','lucas'], faces_path='./data/', n_pictures=20)

Profile found with 16 images.
Profile found with 13 images.
[s]ave a frame ?
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved


Note that we do not use any learning of a feature representation, and apply distances directly on the raw pixels of the segmented faces. Such learning could help in improving the performances.

## Q3

In [4]:
from Large_Scale_Label_Propagation import func as lslp
from importlib import reload
reload(lslp)

<module 'Large_Scale_Label_Propagation.func' from '/home/lucas/Documents/MVA/Graphs/TP3/code_material/Large_Scale_Label_Propagation/func.py'>

In [5]:
lslp.iterative_hfs(niter = 20)

(array([1., 1., 1., ..., 2., 2., 2.]), 0.9998666666666667)